In [1]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
from collections import deque

In [2]:
model1 = load_model('devanagari_model.h5')


In [3]:
letter_count = {0: 'CHECK', 1: '01_ka', 2: '02_kha', 3: '03_ga', 4: '04_gha', 5: '05_kna', 6: 'character_06_cha',
                    7: '07_chha', 8: '08_ja', 9: '09_jha', 10: '10_yna',
                    11: '11_taamatar',
                    12: '12_thaa', 13: '13_daa', 14: '14_dhaa', 15: '15_adna', 16: '16_tabala', 17: '17_tha',
                    18: '18_da',
                    19: '19_dha', 20: '20_na', 21: '21_pa', 22: '22_pha',
                    23: '23_ba',
                    24: '24_bha', 25: '25_ma', 26: '26_yaw', 27: '27_ra', 28: '28_la', 29: '29_waw', 30: '30_motosaw',
                    31: '31_petchiryakha',32: '32_patalosaw', 33: '33_ha',
                    34: '34_chhya', 35: '35_tra', 36: '36_gya', 37: 'CHECK'}

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    print("processed: " + str(processed.shape))
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 32
    image_y = 32
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
cv2.__version__


'4.1.2'

In [6]:
blueLower = np.array([100, 60, 60])
blueUpper = np.array([255, 255, 255])


kernel = np.ones((5, 5), np.uint8)

blackboard = np.zeros((480,640,3), dtype=np.uint8)
alphabet = np.zeros((200, 200, 3), dtype=np.uint8)

points = deque(maxlen=512)

index = 0

In [10]:
    camera = cv2.VideoCapture(0)
    Lower_green = np.array([110, 50, 50])
    Upper_green = np.array([130, 255, 255])

    pred_class=0
    pts = deque(maxlen=512)
    blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
    digit = np.zeros((200, 200, 3), dtype=np.uint8)
    while True:
        (grabbed, frame) = camera.read()
        frame = cv2.flip(frame, 1)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        blueMask = cv2.inRange(hsv, blueLower, blueUpper)
        blueMask = cv2.erode(blueMask, kernel, iterations=2)
        blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
        blueMask = cv2.dilate(blueMask, kernel, iterations=1)
#         thresh = cv2.threshold(blueMask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        
        
#         imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#         mask = cv2.inRange(imgHSV, blueLower, blueUpper)
#         blur = cv2.medianBlur(mask, 15)
#         blur = cv2.GaussianBlur(blur, (5, 5), 0)
#         thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
#         cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        

        (cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
                                     cv2.CHAIN_APPROX_SIMPLE)

        center = None

        if len(cnts) > 0:
            cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
            ((x, y), radius) = cv2.minEnclosingCircle(cnt)
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)

            M = cv2.moments(cnt)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

            points.appendleft(center)
#             for i in range(1, len(pts)):
#                 if pts[i - 1] is None or pts[i] is None:
#                     continue
#                 cv2.line(blackboard, pts[i - 1], pts[i], (255, 255, 255), 10)
#                 cv2.line(img, pts[i - 1], pts[i], (0, 0, 255), 5)

        elif len(cnts) == 0:
            if len(points) != 0:
                blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
                blur1 = cv2.medianBlur(blackboard_gray, 15)
                blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
                thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
                if len(blackboard_cnts) >= 1:
                    cnt = sorted(blackboard_cnts, key = cv2.contourArea, reverse = True)[0]

                    if cv2.contourArea(cnt) > 1000:
                        x, y, w, h = cv2.boundingRect(cnt)
                        digit = blackboard_gray[y:y + h, x:x + w ]

                        pred_probab, pred_class = keras_predict(model1,digit)
                        print(pred_class, pred_probab)

                        points = deque(maxlen=512)
                        blackboard = np.zeros((480, 640, 3), dtype=np.uint8)

              
        for i in range(1, len(points)):
            if points[i - 1] is None or points[i] is None:
                continue
            cv2.line(frame, points[i - 1], points[i], (0, 0, 0), 2)
            cv2.line(blackboard, points[i - 1], points[i], (255, 255, 255), 8)

        cv2.putText(frame, "Conv Network : " + str(letter_count[pred_class]), (10, 410), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255, 255, 255), 2)

        cv2.imshow("alphabets Recognition Real Time", frame)        
    
    

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

processed: (1, 32, 32, 1)
16 0.42698237
processed: (1, 32, 32, 1)
2 0.24205132
processed: (1, 32, 32, 1)
9 0.2436364
processed: (1, 32, 32, 1)
1 0.8451433
processed: (1, 32, 32, 1)
34 0.5284825
processed: (1, 32, 32, 1)
16 0.13476244
processed: (1, 32, 32, 1)
20 0.2536798
processed: (1, 32, 32, 1)
2 0.22555892
processed: (1, 32, 32, 1)
19 0.2658372
processed: (1, 32, 32, 1)
33 0.18594147
processed: (1, 32, 32, 1)
1 0.36898658
processed: (1, 32, 32, 1)
18 0.3929831
processed: (1, 32, 32, 1)
27 0.3238903


In [10]:
pred_probab, pred_class = keras_predict(model1, np.zeros((32, 32, 1), dtype=np.uint8))
str(letter_count[pred_class])

processed: (1, 32, 32, 1)


'27_ra'

In [11]:
camera.release()